In [ ]:
import sys
sys.path.append('..')
import json
import re
from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api.formatters import JSONFormatter

In [ ]:
def open_file(filepath):
    with open(filepath, 'r', encoding='utf-8') as infile:
        return infile.read()


def save_file(content, filepath):
    with open(filepath, 'w', encoding='utf-8') as outfile:
        outfile.write(content)

In [ ]:
def get_transcript(video_id):
    if not video_id:
        raise Exception('Video ID not found')

    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=['en'])

        formatter = JSONFormatter()
        text = formatter.format_transcript(transcript)
        # text = re.sub('\s+', ' ', text).replace('--', '')
        return text

    except Exception as e:
        raise Exception('Could not download the transcript')

In [ ]:
def slice_transcript(transcript, end, start=0):             # example: start | end = 652 (seconds)
    if start > end[0]: raise Exception('Start is ahead of End')

    text = []
    for obj in transcript:
        if start > obj['start']: continue
        if obj['start'] > end:
            return re.sub('\s+', ' ', ' '.join(text))
        text.append(obj['text'])


def slice_transcript(transcript, end=list(), start=list()):     # example: end=list[34.56, 66.45]; start=65.32 (seconds)
    if not len(end): raise Exception('Missing parameter: end')
    if start[0] > end[0]: raise Exception('Start is ahead of End')

    chapters = []
    text = []
    checkpoint = 0
    for obj in transcript:
        if start[0] > obj['start']: continue
        if checkpoint >= len(end): return chapters

        if obj['start'] > end[checkpoint]:
            chapters.append((start[checkpoint], re.sub('\s+', ' ', ' '.join(text))))
            text.clear()
            checkpoint += 1
        text.append(obj['text'])

    return chapters


In [ ]:
def time_to_seconds(time_str):
    parts = [int(part) for part in time_str.split(":")]
    if len(parts) == 2:
        return parts[0] * 60 + parts[1]
    elif len(parts) == 3:
        return parts[0] * 3600 + parts[1] * 60 + parts[2]
    else:
        raise ValueError("Invalid time format: " + time_str)

def transform_chapter_timestamp(chapters):
    '''
    This function expects chapter with timestamp as string. The last item should be the 'End' timestamp, indicating the end of the video.
        ["00:00 Chapter Title"]
        ["04:15 Chapter Title"]
        ["07:21 Chapter Title"]
        ["10:31 End"]
    '''

    chapter_list = [line.strip().split(" ", 1) for line in chapters]

    # chapter titles
    chapter_titles = [chapter[1] for chapter in chapter_list]

    # chapter start ts
    chapter_starts = []
    for chapter in chapter_list:
        if chapter[0][0].isdigit():
            chapter_starts.append(time_to_seconds(chapter[0]))
        else:
            chapter_starts.append(time_to_seconds(chapter[0][1:-1]))

    # chapter end & start ts
    chapter_ends = chapter_starts[1:]
    chapter_starts = chapter_starts[:-1]    # exclude the 'End' timestamp

    return list(zip(chapter_starts, chapter_ends, chapter_titles))

# Action

In [ ]:
video_id = 'CEee7dAk25c'

chapters = '''
(1:57) Overview of the SVB collapse and bank run
(17:53) Who or what is to blame? Debating venture debt
(37:11) Contagion risk, second- and third-order effects, government backstops
(1:00:36) What does this mean for the VC industry? Silicon Valley panic cycle, advice for founders
(1:29:00) End
'''

In [ ]:
chapter_list = chapters.strip().split("\n")
chapter_ts = transform_chapter_timestamp(chapter_list)

ch_starts = [ch[0] for ch in chapter_ts]
ch_ends = [ch[1] for ch in chapter_ts]
print('start:', ch_starts, 'end:', ch_ends)

In [ ]:
max_prompt_len = 15000
# download transcript
transcript = json.loads(get_transcript(video_id))
save_file(json.dumps(transcript), f'./chapters/transcript_{video_id}.txt')

# multiple slice
transcripts = slice_transcript(transcript, ch_ends, ch_starts)

for chapter in transcripts:
    prompt = open_file('prompt_chapter_summary.txt').replace('<<TEXT>>', chapter[1])
    if len(prompt) > max_prompt_len:
        prompt = prompt[:max_prompt_len]
    save_file(prompt, f'./chapters/chapter_{video_id}_{chapter[0]}.txt')